# Configuring and Using Dataset Features 

This notebook shows model developers how they can learn more about ... These instructions include the code required to:

- AAA...
- BBB...

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::


This notebook requires an OpenAI API secret key to run. If you don't have one, visit [API keys](https://platform.openai.com/account/api-keys) on OpenAI's site to create a new key for yourself. Note that API usage charges may apply.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [41]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Foundation Model - Text Sentiment Analysis`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [42]:
## Replace this placeholder with the code snippet from your own project ##

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)
  

2023-10-30 10:33:45,276 - INFO(validmind.api_client): Connected to ValidMind. Project: [Flowcast-2] Collection Scorecard - Initial Validation (clo4az95r00843wy6admynvua)


## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [43]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [44]:
vm.preview_template()

2023-10-30 10:33:45,349 - ERROR(validmind.tests): Unable to load test validmind.model_validation.flowcast.PredictProbHistogram. No module named 'validmind.tests.model_validation.flowcast.PredictProbHistogram'


Accordion(children=(Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), HTML(value='<p…

## Load the sample dataset

In [45]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn as demo_dataset

# You can also try a different dataset with:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

In [46]:
demo_dataset.class_labels

{'0': 'Did not exit', '1': 'Exited'}

In [47]:
demo_dataset.target_column

'Exited'

In [48]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
7995,800,France,Female,29,2,0.00,2,0,0,167773.55,0
7996,771,France,Male,39,5,0.00,2,1,0,96270.64,0
7997,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
7998,709,France,Female,36,7,0.00,1,0,1,42085.58,1


In [49]:
feature_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

### Add segment columns to dataset

In [50]:
import numpy as np

# Function to add Product column
def add_product_column(dataframe):
    conditions = [
        (dataframe['CreditScore'] < 650),
        (dataframe['CreditScore'] >= 650) & (dataframe['CreditScore'] <= 750),
        (dataframe['CreditScore'] > 750)
    ]
    choices = ['Product A', 'Product B', 'Product C']
    dataframe['Product'] = np.select(conditions, choices, default='Product A')
    return dataframe

In [51]:
# df = add_product_column(df)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
7995,800,France,Female,29,2,0.00,2,0,0,167773.55,0
7996,771,France,Male,39,5,0.00,2,1,0,96270.64,0
7997,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
7998,709,France,Female,36,7,0.00,1,0,1,42085.58,1


### Initialize a ValidMind dataset object

Before you can run a test suite, which are just a collection of tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module. 

This function takes a number of arguments: 

- `dataset` — the raw dataset that you want to analyze
- `target_column` — the name of the target column in the dataset 
- `class_labels` — the list of class labels used for classification model training

In [52]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels,
    feature_columns=feature_columns
)

2023-10-30 10:33:45,766 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


Importing client.py from: /Users/juanvalidmind/Dev/github/validmind/developer-framework/validmind/client.py


### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Initialize XGBoost classifier: Creates an `XGBClassifier` object with early stopping rounds set to 10.
- Set evaluation metrics: Specifies metrics for model evaluation as "error," "logloss," and "auc."
- Fit the model: Trains the model on `x_train` and `y_train` using the validation set `(x_val, y_val)`. Verbose output is disabled.

In [53]:
train_val_df, test_df = train_test_split(df, test_size=0.20)
train_df, validation_df = train_test_split(train_val_df, test_size=0.25)

X_train = train_df[feature_columns] 
y_train = train_df[demo_dataset.target_column]
X_val = validation_df[feature_columns] 
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

### Initialize the training and test datasets

With the datasets ready, you can now initialize the training and test datasets (`train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [54]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns
)

2023-10-30 10:33:45,971 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-10-30 10:33:45,990 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


Importing client.py from: /Users/juanvalidmind/Dev/github/validmind/developer-framework/validmind/client.py
Importing client.py from: /Users/juanvalidmind/Dev/github/validmind/developer-framework/validmind/client.py


### Initialize a model object 

Additionally, you need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [55]:
vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

### Missing values by segment and dataset

In [56]:
import pandas as pd

def missing_values_by_segment(datasets, segments, dataset_names=None):
    if dataset_names is None:
        dataset_names = [f"Dataset {i+1}" for i in range(len(datasets))]
    else:
        if len(datasets) != len(dataset_names):
            raise ValueError("Length of dataset_names must match length of datasets")
        if not all(isinstance(name, str) for name in dataset_names):
            raise TypeError("All dataset names must be strings")
    
    if not all(isinstance(dataset, pd.DataFrame) for dataset in datasets):
        raise TypeError("All datasets must be pandas DataFrames")
    if not all(isinstance(segment, str) for segment in segments):
        raise TypeError("All segments must be strings")

    results = []
    for dataset, dataset_name in zip(datasets, dataset_names):
        for segment in segments:
            if segment not in dataset.columns:
                results.append({'Dataset': dataset_name, 'Segment': segment, 'Segment Value': 'N/A', 'Missing Values': 'N/A', 'Total Values': 'N/A', 'Missing Percentage': 'N/A'})
            else:
                total_values = len(dataset)
                for value in dataset[segment].dropna().unique():
                    subset = dataset[dataset[segment] == value]
                    missing_values = subset[segment].isnull().sum()
                    missing_percentage = (missing_values / total_values) * 100
                    results.append({'Dataset': dataset_name, 'Segment': segment, 'Segment Value': value, 'Missing Values': missing_values, 'Total Values': total_values, 'Missing Percentage': missing_percentage})
                
    return pd.DataFrame(results)

In [57]:
datasets = [df, train_df, test_df]
segments = ['Geography', 'Gender']
results = missing_values_by_segment(
    datasets=datasets, 
    segments = segments, 
    dataset_names=['Full Dataset', 'Train', 'Test'])
display(results)

,Dataset,Segment,Segment Value,Missing Values,Total Values,Missing Percentage
0,Full Dataset,Geography,France,0,8000,0.0
1,Full Dataset,Geography,Spain,0,8000,0.0
2,Full Dataset,Geography,Germany,0,8000,0.0
3,Full Dataset,Gender,Female,0,8000,0.0
4,Full Dataset,Gender,Male,0,8000,0.0
5,Train,Geography,Spain,0,4800,0.0
6,Train,Geography,France,0,4800,0.0
7,Train,Geography,Germany,0,4800,0.0
8,Train,Gender,Female,0,4800,0.0
9,Train,Gender,Male,0,4800,0.0


In [60]:
import pandas as pd
import numpy as np

def descriptive_stats_by_segment(datasets, segments, dataset_names=None):
    if dataset_names is None:
        dataset_names = [f"Dataset {i+1}" for i in range(len(datasets))]
    elif len(datasets) != len(dataset_names):
        raise ValueError("Length of dataset_names must match length of datasets")

    if not all(isinstance(dataset, pd.DataFrame) for dataset in datasets):
        raise TypeError("All datasets must be pandas DataFrames")
    if not all(isinstance(segment, str) for segment in segments):
        raise TypeError("All segments must be strings")

    results = []
    for dataset, dataset_name in zip(datasets, dataset_names):
        for segment in segments:
            if segment not in dataset.columns:
                results.append({'Dataset': dataset_name, 'Segment': segment, 'Segment Value': 'N/A', 'Count': 'N/A', 'Mean': 'N/A', 'Std': 'N/A', 'Min': 'N/A', '25%': 'N/A', '50%': 'N/A', '75%': 'N/A', 'Max': 'N/A'})
            else:
                for value in dataset[segment].dropna().unique():
                    subset = dataset[dataset[segment] == value]
                    numeric_data = subset.select_dtypes(include=np.number)

                    if not numeric_data.empty:
                        stats = numeric_data.stack().describe()
                        results.append({
                            'Dataset': dataset_name, 
                            'Segment': segment, 
                            'Segment Value': value, 
                            'Count': stats['count'],
                            'Mean': stats['mean'], 
                            'Std': stats['std'], 
                            'Min': stats['min'], 
                            '25%': stats['25%'], 
                            '50%': stats['50%'], 
                            '75%': stats['75%'], 
                            'Max': stats['max']
                        })

    return pd.DataFrame(results)


In [61]:
df_stats = descriptive_stats_by_segment(
    datasets=datasets,
    segments = segments, 
    dataset_names=['Full Dataset', 'Train', 'Test'])
display(df_stats)

,Dataset,Segment,Segment Value,Count,Mean,Std,Min,25%,50%,75%,Max
0,Full Dataset,Geography,France,36090.0,18037.056242,45031.856560,0.0,1.0,2.0,586.0,238387.56
1,Full Dataset,Geography,Spain,17829.0,17894.078686,44876.928944,0.0,1.0,2.0,587.0,250898.09
2,Full Dataset,Geography,Germany,18081.0,24632.497213,50833.899379,0.0,1.0,5.0,718.0,214346.96
3,Full Dataset,Gender,Female,32436.0,19652.005845,46523.576334,0.0,1.0,2.0,623.0,238387.56
4,Full Dataset,Gender,Male,39564.0,19662.789637,46678.332182,0.0,1.0,2.0,620.0,250898.09
5,Train,Geography,Spain,10611.0,17990.722988,45219.730412,0.0,1.0,2.0,579.0,250898.09
6,Train,Geography,France,22050.0,17794.458972,44694.990605,0.0,1.0,2.0,579.0,212314.03
7,Train,Geography,Germany,10539.0,24654.324479,50817.525736,0.0,1.0,5.0,720.0,214346.96
8,Train,Gender,Female,19089.0,19521.605871,46332.876713,0.0,1.0,2.0,625.0,216109.88
9,Train,Gender,Male,24111.0,19511.898020,46595.655618,0.0,1.0,2.0,613.0,250898.09
